# Class Imbalances and Software Quality
* [Class Imbalances](#classimbheader)
* [Software Quality](#softwarequalityheader)
# Class Imbalances<a name="classimbheader"></a>
## Background
## 
## References
1. Nitesh V. Chawla, Kevin W. Bowyer, Lawrence O. Hall, W. Philip Kegelmeyer: SMOTE: Synthetic Minority Over-Sampling Technique. Journal of Artificial Intelligence Reseach 16 (2002) 321-357. [https://arxiv.org/pdf/1106.1813.pdf](https://arxiv.org/pdf/1106.1813.pdf)

2. Hui Han, Wen-Yuan Wang, Bing-Huan Mao: Borderline-SMOTE: A New Over-Sampling Method in Imbalanced Data Sets Learning. Advances in Intelligent Computing, International Conference on Intelligent Computing (2005). [https://sci2s.ugr.es/keel/keel-dataset/pdfs/2005-Han-LNCS.pdf](https://sci2s.ugr.es/keel/keel-dataset/pdfs/2005-Han-LNCS.pdf)

3. Guillaume Lemaitre, Fernando Nogueira, Christos K. Aridas: Imbalanced-learn: A Python Toolbox to Tackle the Curse of Imbalanced Datasets in Machine Learning. Journal of Machine Learning Research 18 (2017) 1-5. [http://www.jmlr.org/papers/volume18/16-365/16-365.pdf](http://www.jmlr.org/papers/volume18/16-365/16-365.pdf)

# Software Quality<a name="softwarequalityheader"></a>
sdfsadfasdf
dafsasdfasdfasdf
sdfasdfasdf